![alt text for screen readers](./horizontal_ORIGINAL.jpg "Bladesight").
## Chapter 7: Blade Vibration Theory 

Run this cell first 👇

In [2]:
import pandas as pd
from typing import List, Tuple
from scipy.integrate import cumulative_trapezoid 
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from typing import Callable, List
from tqdm import tqdm
import os

import pathlib

import numpy as np
from numba import njit
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from plotly.subplots import make_subplots

pio.templates.default = "ggplot2"

In [ ]:
def periodic_step_function(
        theta : np.ndarray, 
        theta_min : float, 
        theta_max : float,
        amplitude : float
    ) -> np.ndarray:
    """
    A periodic step function.

    Args:
        theta (np.ndarray): The independent variable.
        theta_min (float): The minimum value of the independent variable.
        theta_max (float): The maximum value of the independent variable.
        amplitude (float): The amplitude of the step function.

    Returns:
        np.ndarray: The periodic step function.
    """
    return amplitude * (np.heaviside(theta % (2*np.pi) - theta_min, 1) - np.heaviside(theta % (2*np.pi) - theta_max, 1))
theta_min = 0
theta_max = np.pi/10
amplitude = 1
theta = np.linspace(-2*np.pi, 2 * np.pi * 5, 3000)
force = periodic_step_function(theta, theta_min, theta_max, amplitude)
theta = np.round(theta, 3)
force = np.round(force, 3)
fig = px.line(x=theta, y=force, labels={"x": r"$\theta$", "y": r"$f(\theta)$"})
# Update the x axis to be in multiples of pi
fig.update_xaxes(
    tickmode="array", 
    tickvals=np.arange(0, 2*np.pi*3, np.pi/2), 
    ticktext=[
        "0", 
        r"$\pi/2$", 
        r"$\pi$", 
        r"$1\frac{1}{2}\pi$", 
        r"$2\pi$", 
        r"$2\frac{1}{2}\pi$",
        r"$3\pi$", 
        r"$3\frac{1}{2}\pi$",
        r"$4\pi$",
        r"$4\frac{1}{2}\pi$",
        r"$5\pi$",
        r"$5\frac{1}{2}\pi$",
    ]
)
# Update the x limit to -1 to 7
fig = fig.update_xaxes(range=[-1, 2.7*2*np.pi])
fig.write_html("plot_periodic_step_function.html", include_plotlyjs="cdn", full_html=False)
fig

In [ ]:
def calculate_positite_step_function_fourier_transform(
    force : np.ndarray,
    theta : np.ndarray,
    n : int,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate the Fourier coefficients of a positive step function.

    Args:
        force (np.ndarray): The force.
        theta (np.ndarray): The independent variable.
        n (int): The number of Fourier coefficients to calculate.

    Returns:
        Tuple[np.ndarray, np.ndarray]: The Fourier coefficients and the independent variable.
    """
    # Calculate the Fourier coefficients
    fourier_coefficients = 2*np.fft.fft(force)/len(theta)
    # Calculate the independent variable
    frequency = np.fft.fftfreq(len(theta), d=theta[1]/(2*np.pi) - theta[0]/(2*np.pi))
    # Return the Fourier coefficients and the independent variable
    return frequency[:n], fourier_coefficients[:n]

theta = np.linspace(0, 2 * np.pi * 50, 30000)
force = periodic_step_function(theta, theta_min, theta_max, amplitude)
# Apply a Hanning window to the force
frequencies, fourier_coefficients = calculate_positite_step_function_fourier_transform(force, theta, 600)

In [ ]:
fig = px.line(
    x=np.round(frequencies,2), 
    y=np.round(np.abs(fourier_coefficients),2), 
    labels={"x": "EO", "y": "Magnitude"}, 
)
fig.write_html("plot_periodic_step_function_fourier_transform.html", include_plotlyjs="cdn", full_html=False)
fig

## Fig: Campbell Diagram

In [ ]:
nat_freq_1 = interp1d([0,2000,3000],[120,135,149], kind="quadratic")
nat_freq_2 = interp1d([0,2000,3000],[275,301,340], kind="quadratic")
nat_freq_3 = interp1d([0,2000,3000],[375,425,500], kind="quadratic")
omegas_campbell = np.round(np.linspace(1000, 3000, 3000),2)
EO_vs_nat_freq_1 = interp1d(nat_freq_1(omegas_campbell), omegas_campbell)
EO_vs_nat_freq_2 = interp1d(nat_freq_2(omegas_campbell), omegas_campbell)
EO_vs_nat_freq_3 = interp1d(nat_freq_3(omegas_campbell), omegas_campbell)

fig = go.Figure()
fig.add_trace(go.Scatter(x=omegas_campbell, y=np.round(nat_freq_1(omegas_campbell),2), name="Mode 1"))
fig.add_trace(go.Scatter(x=omegas_campbell, y=np.round(nat_freq_2(omegas_campbell),2), name="Mode 2"))
fig.add_trace(go.Scatter(x=omegas_campbell, y=np.round(nat_freq_3(omegas_campbell),2), name="Mode 3"))
EOs = [2,3,4,5,6,7,8,9, 10,11,12]
for EO in EOs:
    fig = fig.add_trace(
        go.Scatter(
            x=[omegas_campbell[0], omegas_campbell[-1]], 
            y=[EO*omegas_campbell[0]/60, EO*omegas_campbell[-1]/60], 
            mode='lines+text',
            line_dash="dash", 
            line_color="black",
            text=["", f"   EO={EO}"],
            textposition="middle right",
            name = f'EO = {EO}',
            showlegend=False,
        ),
    )
crossing_points = []
for nat_freq_func in [nat_freq_1, nat_freq_2, nat_freq_3]:
    for EO in EOs:
        fac = nat_freq_func(omegas_campbell)
        r = fac / (omegas_campbell / 60 * EO)
        flip = np.sign(r - 1)
        d_flip = np.diff(flip)
        if np.sum(d_flip < 0) > 0:
            ix_cross = np.argmin(d_flip)
            omega_cross = omegas_campbell[ix_cross]
            f_cross = float(nat_freq_func(omega_cross))
            crossing_points.append({
                "EO": EO,
                "omega_cross": omega_cross,
                "f_cross": f_cross,
            })

df_crossing_points = pd.DataFrame(crossing_points)
fig = fig.add_trace(
    go.Scatter(
        x=np.round(df_crossing_points["omega_cross"],2),
        y=np.round(df_crossing_points["f_cross"],2),
        mode='markers',
        marker=dict(
            size=15,
            color="black",
            opacity=0.5,
        ),
        name="Potential resonances",
        showlegend=True,
    ),
)

fig.update_xaxes(title_text="Rotor Speed [RPM]")
# Update x axis limits to 1000 to 3050
fig = fig.update_xaxes(range=[1000, 3200])
fig.update_yaxes(title_text="Frequency (Hz)")
fig.write_html("plot_campbell_diagram.html", include_plotlyjs="cdn", full_html=False)
fig

In [ ]:
def get_X(
        omega : np.ndarray, 
        omega_0 : float,  
        zeta: float,
        delta_st: float = 1
    ) -> np.ndarray:
    """
    This function returns the vibration amplitude of 
    the blade vibration.
    
    x(ω) = 	δ_st / sqrt( (1 - r**2) + (2*ζ*r)**2)

    where:

    r = ω/ω_0

    Args:
        omega (np.ndarray): The excitation frequencies in rad/s.
        omega_0 (float): The natural frequency of the blade in rad/s.
        zeta (float): The damping ratio of the blade vibration.
        delta_st (float, optional): The static deflection of the blade 
            in mm. Defaults to 1.

    Returns:
        np.ndarray: The amplitude of the blade vibration in mm.
    """
    r = omega / omega_0
    return delta_st / np.sqrt((1 - r**2)**2 + (2*zeta*r)**2)

def get_phi(omega : np.ndarray, omega_0 : float, zeta: float) -> np.ndarray:
    """Get the phase of the blade vibration.

    φ(ω) = arctan(2*ζ*r /  (1 - r**2))
    
    where:
    r = ω/ω_0

    Args:
        omega (np.ndarray): The excitation frequencies in rad/s.
        omega_0 (float): The natural frequency of the blade in rad/s.
        delta (float): The damping ratio of the blade vibration.

    Returns:
        np.ndarray: The phase of the blade vibration in rad.
    """
    r = omega / omega_0
    return np.arctan2(2 * zeta * r,1 - r**2)

## Fig: SDoF model solution

In [ ]:
# Start a plotly figure with two horizontal subplots
fig = make_subplots(rows=2, cols=1, subplot_titles=("Amplitude", "Phase"))
zetas = [0.01,0.02,0.05,0.2,0.5]
omega_n = 120
omegas = np.round(np.linspace(60 , 180, 1000),2)
amps = {}
phase = {}
for zeta in zetas:
    amps[f"$X(\zeta = {zeta})$"] = np.round(get_X(omegas, omega_n, zeta),2)
    phase[fr"$\phi(\zeta = {zeta})$"] = np.round(get_phi(omegas, omega_n, zeta),2)

for key, value in amps.items():
    fig.add_trace(go.Scatter(x=omegas, y=value, name=key), row=1, col=1)

for key, value in phase.items():
    fig.add_trace(go.Scatter(x=omegas, y=value, name=key), row=2, col=1)


fig.update_xaxes(title_text="Excitation Frequency [Hz]", row=1, col=1)
fig.update_xaxes(title_text="Excitation Frequency [Hz]", row=2, col=1)
fig.update_yaxes(title_text="Amplitude [mm]", row=1, col=1)
fig.update_yaxes(title_text="Phase [rad]", row=2, col=1)
fig.write_html("plot_amplitude_phase.html", include_plotlyjs="cdn", full_html=False)
fig


Fig: Polar blade vibration and sampling

In [ ]:
theta = np.linspace(0,2*np.pi,1000)
omega_n = 120
zeta = 0.05
EO = 6
delta_st = 1
frame_count = 120
measurements = []
sensor_locations = [45, 67, 123, 150] 
sensor_colors = ["red", "green", "magenta","orange"]

logo_image = imageio.imread('symbol_ORIGINAL.png')
logo_small = logo_image[::5, ::5, :]
logo_width = logo_small.shape[1]
logo_height = logo_small.shape[0]

for frame_no, omega_excitation in tqdm(enumerate(np.linspace(80, 160, frame_count))):
    X = get_X(omega_excitation, omega_n, zeta, delta_st)
    r_add = 30
    phi = get_phi(omega_excitation, omega_n, zeta)
    x = X * np.cos(EO * theta + phi)

    # Create a polar plotly figure with the blade vibration
    plot_r = x + r_add
    plot_theta = theta/(2*np.pi) * 360

    f_plot_r = interp1d(plot_theta, plot_r)
    f_plot_x = interp1d(plot_theta, x)

    max_r = plot_r.max()
    fig_polar = go.Figure()

    fig_polar.add_trace(go.Scatterpolar(
        r=plot_r,
        theta=plot_theta,
        name='Blade vibration ($x(t)$)'
    ))
    fig_polar.add_trace(
        go.Barpolar(
            r=[max_r*0.2]*len(sensor_locations),
            base=[max_r*1.4]*len(sensor_locations),
            theta=sensor_locations,
            width=[5]*len(sensor_locations),
            marker_color=sensor_colors,
            marker_line_color=sensor_colors,
            showlegend=True,
            name=r"$\text{Sensors}$",
            text=["Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4"],
        )
    )
    for theta_sensor in sensor_locations:
        # Add a dotted line from max_r*1.4 to the wave position f_plot_r(theta_sensor)
        fig_polar.add_trace(
            go.Scatterpolar(
                r=[max_r*1.4, f_plot_r(theta_sensor)],
                theta=[theta_sensor, theta_sensor],
                mode="lines",
                line=dict(color="green", dash="dot"),
                showlegend=False,
            )
        )
    # Make large opaque black dots at the sensor locations
    fig_polar.add_trace(
        go.Scatterpolar(
            r=f_plot_r(np.array(sensor_locations)),
            theta=sensor_locations,
            mode="markers",
            marker=dict(color="black", size=10, opacity=0.5),
            showlegend=False,
        )
    )
    # Hide the radial axis
    fig_polar.update_polars(radialaxis_visible=False)
    # Add a title 
    fig_polar.update_layout(
        title=rf"$a) \omega = {round(omega_excitation,2)} Hz$"
    )
    # Now plot the linear plot
    fig_linear = go.Figure()
    measurements.append(
        {
            "omega_excitation": omega_excitation,
            r"$x(t) \text{ at Sensor 1}$": f_plot_x(sensor_locations[0]),
            r"$x(t) \text{ at Sensor 2}$": f_plot_x(sensor_locations[1]),
            r"$x(t) \text{ at Sensor 3}$": f_plot_x(sensor_locations[2]),
            r"$x(t) \text{ at Sensor 4}$": f_plot_x(sensor_locations[3]),
        }
    )
    df_measurements = pd.DataFrame(measurements)
    for i_column, column in enumerate(df_measurements.columns[1:]):
        fig_linear.add_trace(
            go.Scatter(
                x=df_measurements["omega_excitation"],
                y=df_measurements[column],
                mode="lines",
                line=dict(color=sensor_colors[i_column]),
                name=column,
            )
        )
    fig_linear.add_trace(
        go.Scatter(
            x=[df_measurements["omega_excitation"].iloc[-1]] * 4,
            y=df_measurements.iloc[-1, 1:],
            mode="markers",
            marker=dict(color=sensor_colors, size=10, opacity=0.5),
            showlegend=False
        )
    )

    fig_linear.update_xaxes(title_text=r"$\text{Excitation Frequency}, EO \cdot \Omega [Hz]$")
    fig_linear.update_yaxes(title_text="$x$ [mm]")
    fig_linear.update_xaxes(range=[80, 160])
    fig_linear.update_yaxes(range=[-12, 12])
    # Update x axis font to 25
    fig_linear.update_layout(
        title=rf"$b) \omega = {round(omega_excitation,2)} Hz$"
    )
    fig_polar.write_image(f"plot_blade_vibration_polar_{str(frame_no).zfill(2)}.png", width=400, height=400, scale=2)
    fig_linear.write_image(f"plot_blade_vibration_linear_{str(frame_no).zfill(2)}.png", width=500, height=400, scale=2)
file_root_polar = "plot_blade_vibration_polar_"
file_root_linear = "plot_blade_vibration_linear_"
images = []
for file_number in range(frame_count):
    file_name_polar = file_root_polar + str(file_number).zfill(2) + ".png"
    file_name_linear = file_root_linear + str(file_number).zfill(2) + ".png"
    polar_image = imageio.imread(file_name_polar)
    linear_image = imageio.imread(file_name_linear)
    # Concatenate the images side by side
    image = np.concatenate((polar_image, linear_image), axis=1)
    # Add the logo
    image[-logo_height-10:-10, 10:logo_width+10, :3] = logo_small[: ,:, :3]
    images.append(image)
    # Now delete the images
    os.remove(file_name_polar)
    os.remove(file_name_linear)
imageio.mimsave('plot_blade_vibration_polar.gif', images)


## Data simulation starts here

First we define the simulation functions 👇

In [ ]:
@njit
def get_gp_covariance(t : np.ndarray, sigma : np.ndarray, l : float):
    """
    This function returns the covariance matrix for the Gaussian process.
    """
    len_t = t.shape[0]
    K = np.zeros((len_t, len_t))
    for i in range(len_t):
        for j in range(len_t):
            k = sigma[i]**2 * np.exp(-1 * (t[i] - t[j])**2/(2*l**2))
            K[i, j] = k
            if i != j:
                K[j, i] = k
    return K

def get_simulated_time_stamps(
        t_sampled : np.ndarray, 
        theta_sampled : np.ndarray, 
        sensor_thetas : List[float], 
        f_amplitude : Callable, 
        f_phase : Callable,
        EO : int,
        R : float,
        B : int
    ):
    """
    This function returns the zero crossings of the OPR.

    Args:
        t_sampled (np.ndarray): Time stamps corresponding to the shaft's angular position
        theta_sampled (np.ndarray): The shaft's angular position
        sensor_thetas (List[float]): The angular positions of the sensors. The length of this
            list corresponds the number of sensors. The position of the sensors
            should be provided in radians.
        f_amplitude (Callable): A function that returns the blade 
            vibration amplitude for a given time stamp
        f_phase (Callable): A function that returns the blade 
            vibration phase for a given time stamp
        EO (int): The engine order
        R (float): The radius of the rotor in mm.
        B (int): The number of blades.

    Returns:
        np.ndarray: The zero crossings of the OPR
        Dict[int, List[float]]: A dictionary containing the time stamps when each sensor
            was crossed.
        Dict[int, Dict[int, List[float]]]: A dictionary containing the time stamps for when 
            each sensor was crossed for each blade.
    """
    len_thetas = theta_sampled.shape[0]
    zero_crossings = [0]
    toas = {i:[] for i in range(len(sensor_thetas))}
    blade_spacing = 2 * np.pi / B
    blade_locations = np.arange(blade_spacing / 2, 2*np.pi, blade_spacing)

    latent_variables = {
        b:{
            s:[] for s in range(len(sensor_thetas))
        } for b in range(len(blade_locations))
    }

    for i in tqdm(range(len_thetas - 1)):
        omega = (theta_sampled[i+1] - theta_sampled[i]) / (t_sampled[i+1] - t_sampled[i])
        if theta_sampled[i] > theta_sampled[i+1]:
            # The OPR has ocurred
            omega = (theta_sampled[i+1] + 2*np.pi - theta_sampled[i]) / (t_sampled[i+1] - t_sampled[i])
            delta_t = (2*np.pi - theta_sampled[i]) / omega
            zero_crossings.append(t_sampled[i] + delta_t)

        for b in range(len(blade_locations)):
            theta_blade = (blade_locations[b] + theta_sampled[i]) % (2*np.pi)
            theta_blade_next = (blade_locations[b] + theta_sampled[i+1]) % (2*np.pi)
            for s in range(len(sensor_thetas)):
                theta_s = sensor_thetas[s]
                if theta_blade < theta_s and theta_blade_next >= theta_s:
                    # A blade has crossed a sensor
                    delta_t = (theta_s - theta_blade) / omega
                    t_non_vibrating = t_sampled[i] + delta_t
                    amplitude = f_amplitude(t_non_vibrating)
                    phase = f_phase(t_non_vibrating)
                    x = (
                        amplitude
                        * np.cos(theta_s * EO - phase)
                    )
                    t_vibrating = t_non_vibrating - x / (R * omega)  
                    toas[s].append(t_vibrating)
                    latent_variables[b][s].append(
                        {
                            "t_non_vibrating": t_non_vibrating, 
                            "x": x,
                            "theta_s" : theta_s,
                            "theta_s * EO" : theta_s * EO,
                            "amplitude" : amplitude,
                            "phase" : phase,
                        }
                    )
    return np.array(zero_crossings), toas, latent_variables

Simulate the shaft speed:

In [11]:
time_start = [0, 150, 180, 210, 360]
speed_start = [1000, 2900, 3000, 2900, 1000]
f_omega = interp1d(
    x = time_start,
    y = speed_start,
    fill_value = "extrapolate",
    kind='quadratic'
)
dt_min = 1 / 50
t = np.arange(0, time_start[-1] + dt_min, dt_min)
omega_raw = f_omega(t)

# Uncomment the below two lines to add some variation to the signals
#var_omega = 0.005 * omega_raw
#K = get_gp_covariance(t, var_omega, dt_min*7)
#m = np.random.multivariate_normal(omega_raw, K, 1).flatten()

f_omega = interp1d(
    x = t,
    y = omega_raw,
    fill_value = "extrapolate"
)
t_new = np.arange(0, time_start[-1] + dt_min/150, dt_min/150)
y_new = f_omega(t_new) / 60 * (2 * np.pi)
theta = cumulative_trapezoid(y_new, t_new, initial=0)

Perform the simulation:

In [12]:
sensor_thetas = np.array([30, 65, 120, 140]) / 360 * 2 * np.pi
theta_wrapped = theta % (2 * np.pi)

fn = 120 # The blade's natural frequency in Hz
omega_n = fn * 2 * np.pi # The blade's natural frequency in rad/s
delta_st = 0.3 # The blade's static vibration amplitude in radians
zeta = 0.01 # The blade's vibration damping ratio
omega_shaft = f_omega(t_new) / 60 * 2 * np.pi # The shaft's angular speed in rad/s 
                                              # for the new time stamps 
EO = 3 # The Engine Order
R = 1000 # The radius of the rotor in mm
Blades = 5
omega_excitation = omega_shaft * EO # The excitation frequency in rad/s

vibration_amplitude = get_X(omega_excitation, omega_n, zeta, delta_st)
vibration_phase = get_phi(omega_excitation, omega_n, zeta)
func_amplitude = interp1d(
    x = t_new,
    y = vibration_amplitude,
    fill_value = "extrapolate"
) # A function that returns the blade vibration amplitude for a given time stamp
func_phase = interp1d(
    x = t_new,
    y = vibration_phase,
    fill_value = "extrapolate"
) # A function that returns the blade vibration phase for a given time stamp

zero_crossings, toas, latent_variables = get_simulated_time_stamps(
    t_new, 
    theta_wrapped, 
    sensor_thetas, 
    func_amplitude, 
    func_phase, 
    EO, 
    R,
    Blades
)
# Save the simulated data to file
np.save("../example_data/zero_crossings.npy", zero_crossings)
for i, arr_toas in toas.items():
    np.save(f"../example_data/toas_{i}.npy", arr_toas)

100%|██████████| 2700000/2700000 [00:54<00:00, 49462.74it/s]


In [13]:
vibration_amplitude = get_X(omega_excitation, omega_n, zeta, delta_st)

In [14]:
vibration_amplitude.max() / 1000

0.01500075005349955

In [ ]:
omega_shaft = f_omega(t_new) / 60 * 2 * np.pi

In [ ]:
omega_excitation.max()

In [ ]:
df_lat_1

In [22]:
df_lat_1 = pd.DataFrame(latent_variables[0][0])
df_lat_1.to_csv("../example_data/df_lat_1.csv")
df_lat_2 = pd.DataFrame(latent_variables[0][1])
df_lat_2.to_csv("../example_data/df_lat_2.csv")
df_lat_3 = pd.DataFrame(latent_variables[0][2])
df_lat_3.to_csv("../example_data/df_lat_3.csv")
df_lat_4 = pd.DataFrame(latent_variables[0][3])
df_lat_4.to_csv("../example_data/df_lat_4.csv")

fig = go.Figure()
fig.add_trace(go.Scattergl(x=df_lat_1['t_non_vibrating'], y=df_lat_1['x'], mode='markers+lines', name='Sensor 1'))
fig.add_trace(go.Scattergl(x=df_lat_2['t_non_vibrating'], y=df_lat_2['x'], mode='markers+lines', name='Sensor 2'))
fig.add_trace(go.Scattergl(x=df_lat_3['t_non_vibrating'], y=df_lat_3['x'], mode='markers+lines', name='Sensor 3'))
fig.add_trace(go.Scattergl(x=df_lat_4['t_non_vibrating'], y=df_lat_4['x'], mode='markers+lines', name='Sensor 4'))
fig.update_layout(
    title="True tip deflections",
    xaxis_title="Time (s)",
    yaxis_title="Tip displacement (mm)",
    legend_title="Sensors",
    font=dict(
        size=18,
    )
)


fig.show()

## Figure: Shaft speed and blade vibration amplitude

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=np.round(t,2), y=np.round(omega_raw,2), mode='lines', name='Shaft speed'), secondary_y=False)

df_lat_1 = pd.DataFrame(latent_variables[0][0])
df_lat_2 = pd.DataFrame(latent_variables[0][1])
df_lat_3 = pd.DataFrame(latent_variables[0][2])
df_lat_4 = pd.DataFrame(latent_variables[0][3])

fig.add_trace(go.Scatter(x=np.round(df_lat_1['t_non_vibrating'][::3],2), y=np.round(df_lat_1['x'][::3],2), mode='markers+lines', name=r'$x \text{ Sensor 1}$'), secondary_y=True)
fig.add_trace(go.Scatter(x=np.round(df_lat_2['t_non_vibrating'][::3],2), y=np.round(df_lat_2['x'][::3],2), mode='markers+lines', name=r'$x \text{ Sensor 2}$'), secondary_y=True)
fig.add_trace(go.Scatter(x=np.round(df_lat_3['t_non_vibrating'][::3],2), y=np.round(df_lat_3['x'][::3],2), mode='markers+lines', name=r'$x \text{ Sensor 3}$'), secondary_y=True)
fig.add_trace(go.Scatter(x=np.round(df_lat_4['t_non_vibrating'][::3],2), y=np.round(df_lat_4['x'][::3],2), mode='markers+lines', name=r'$x \text{ Sensor 4}$'), secondary_y=True)
fig.update_layout(
    title="Simulated shaft speed and blade vibration",
    xaxis_title="Time (s)",
    yaxis_title="Tip displacement (mm)",
    legend_title="Sensors"
)

fig.update_yaxes(title_text="Shaft speed [RPM]", secondary_y=False)
fig.update_yaxes(title_text="Tip displacement (mm)", secondary_y=True)
# Hide the second axis grid
fig.update_yaxes(showgrid=False, secondary_y=True)
# Set the second axis range to between -18 and 18
fig.update_yaxes(range=[-18, 18], secondary_y=True)
fig.write_html("simulated_shaft_speed_and_blade_vibration.html", include_plotlyjs="cdn", full_html=False)
fig.show()

## The Blade Tip Timing Processing Starts Here 👇

We start with defining all the functions that we've built up

In [ ]:
from intro_to_btt_functions import (
    calculate_shaft_speed,
    calculate_aoas,
    calculate_Q,
    create_polar_histogram,
    group_blades,
    merge_blades,
    align_blades_over_probes,
    order_domain_zero,
    filter_AoAs
)

## Now we go through the processing steps


In [ ]:
zero_crossings = np.load('../example_data/zero_crossings.npy')
toas = {}
for i in range(4):
    toas[i] = np.load(f'../example_data/toas_{i}.npy')
B = 5
sensor_thetas_deg = np.array([45, 67, 123, 150])
S = len(sensor_thetas_deg)
df_opr = pd.DataFrame(
    {
        "opr_zero_crossing_times": zero_crossings
    }
)
df_probe_1 = pd.DataFrame({"ToA": toas[0]})
df_probe_2 = pd.DataFrame({"ToA": toas[1]})
df_probe_3 = pd.DataFrame({"ToA": toas[2]})
df_probe_4 = pd.DataFrame({"ToA": toas[3]})

# Calculate the AoAs for each probe
columns = [
    "n", 
    "zero_crossing_start", 
    "zero_crossing_end", 
    "ToA",
    "omega", 
    "AoA"
]
df_aoas_1 = pd.DataFrame(
    calculate_aoas(
        df_opr["opr_zero_crossing_times"].values, 
        df_probe_1["ToA"].values
    ), columns=columns
).query("n != -1").reset_index(drop=True)
df_aoas_1['AoA_deg'] = df_aoas_1['AoA'] * 180 / np.pi

df_aoas_2 = pd.DataFrame(
    calculate_aoas(
        df_opr["opr_zero_crossing_times"].values,
        df_probe_2["ToA"].values
    ), columns=columns
).query("n != -1").reset_index(drop=True)
df_aoas_2['AoA_deg'] = df_aoas_2['AoA'] * 180 / np.pi

df_aoas_3 = pd.DataFrame(
    calculate_aoas(
        df_opr["opr_zero_crossing_times"].values,
        df_probe_3["ToA"].values
    ), columns=columns
).query("n != -1").reset_index(drop=True)
df_aoas_3['AoA_deg'] = df_aoas_3['AoA'] * 180 / np.pi

df_aoas_4 = pd.DataFrame(
    calculate_aoas(
        df_opr["opr_zero_crossing_times"].values,
        df_probe_4["ToA"].values
    ), columns=columns
).query("n != -1").reset_index(drop=True)
df_aoas_4['AoA_deg'] = df_aoas_4['AoA'] * 180 / np.pi

# Aligning the blades
optimal_d_thetas = []
for df_aoa in [df_aoas_1, df_aoas_2, df_aoas_3, df_aoas_4]:
    d_thetas = []
    Qs = []
    for d_theta in np.linspace(0, 360/B, 100):
        d_thetas.append(d_theta)
        Qs.append(
            calculate_Q(
                df_aoa,
                d_theta, 
                B
            )
        )
    optimal_d_thetas.append(d_thetas[np.argmin(Qs)])

# Group the blades according to the optimal d_thetas
probe_1_blade_groups, probe_1_blade_median_aoas = group_blades(df_aoas_1, optimal_d_thetas[0], B)
probe_2_blade_groups, probe_2_blade_median_aoas = group_blades(df_aoas_2, optimal_d_thetas[1], B)
probe_3_blade_groups, probe_3_blade_median_aoas = group_blades(df_aoas_3, optimal_d_thetas[2], B)
probe_4_blade_groups, probe_4_blade_median_aoas = group_blades(df_aoas_4, optimal_d_thetas[3], B)

# Blade positions
sensor_positions_from_first = sensor_thetas_deg[1:] - sensor_thetas_deg[0]
sensor_blade_positions = [
    probe_1_blade_median_aoas,
    probe_2_blade_median_aoas,
    probe_3_blade_median_aoas,
    probe_4_blade_median_aoas
]
rotor_blade_indices = align_blades_over_probes(
    sensor_positions_from_first,
    sensor_blade_positions,
    5
)
print(rotor_blade_indices)

df_blade_1 = merge_blades([
    probe_1_blade_groups[rotor_blade_indices[0][0]],
    probe_2_blade_groups[rotor_blade_indices[0][1]],
    probe_3_blade_groups[rotor_blade_indices[0][2]],
    probe_4_blade_groups[rotor_blade_indices[0][3]]
])

df_blade_2 = merge_blades([
    probe_1_blade_groups[rotor_blade_indices[1][0]],
    probe_2_blade_groups[rotor_blade_indices[1][1]],
    probe_3_blade_groups[rotor_blade_indices[1][2]],
    probe_4_blade_groups[rotor_blade_indices[1][3]]
])

df_blade_3 = merge_blades([
    probe_1_blade_groups[rotor_blade_indices[2][0]],
    probe_2_blade_groups[rotor_blade_indices[2][1]],
    probe_3_blade_groups[rotor_blade_indices[2][2]],
    probe_4_blade_groups[rotor_blade_indices[2][3]]
])

df_blade_4 = merge_blades([
    probe_1_blade_groups[rotor_blade_indices[3][0]],
    probe_2_blade_groups[rotor_blade_indices[3][1]],
    probe_3_blade_groups[rotor_blade_indices[3][2]],
    probe_4_blade_groups[rotor_blade_indices[3][3]]
])

df_blade_5 = merge_blades([
    probe_1_blade_groups[rotor_blade_indices[4][0]],
    probe_2_blade_groups[rotor_blade_indices[4][1]],
    probe_3_blade_groups[rotor_blade_indices[4][2]],
    probe_4_blade_groups[rotor_blade_indices[4][3]]
])

## Lets check the alignment 📈

Confirm that the green and black histogram bars are on top of one another. 

In [ ]:
create_polar_histogram(
    df_aoas_1,
    optimal_d_thetas[0],
    B,
)

In [ ]:
df_blade_1_order_zeroed = order_domain_zero(df_blade_1, 4, 1)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=df_blade_1_order_zeroed["n"].iloc[::3], 
        y=np.round(df_blade_1_order_zeroed["omega"].iloc[::3]*60/(2*np.pi),2), 
        mode='lines', 
        name='Shaft speed'
    ), secondary_y=False
)

for i in range(1,5):
    fig.add_trace(
        go.Scatter(
            x=df_blade_1_order_zeroed["n"].iloc[::3],
            y=np.round(df_blade_1_order_zeroed[f"AoA_{i}"].iloc[::3]*1000,2),
            mode='markers+lines',
            name=r'$x \text{ Sensor 1}$'
        ), secondary_y=True
    )


fig.update_yaxes(title_text="Shaft speed [RPM]", secondary_y=False)
fig.update_yaxes(title_text="Tip displacement (mm)", secondary_y=True)
# Hide the second axis grid
fig.update_yaxes(showgrid=False, secondary_y=True)
# Set the second axis range to between -18 and 18
fig.update_yaxes(range=[-18, 18], secondary_y=True)
fig.write_html("simulated_aoas_calculated.html", include_plotlyjs="cdn", full_html=False)
fig.show()

In [ ]:
px.line(df_blade_1_order_zeroed, x='n', y=["AoA_1", "AoA_2", "AoA_3", "AoA_4"] , title='Blade 1')